# Regularization
Authors: Brian Stucky, Carson Andorf

## 1. Introduction

Up to this point, we improved our machine learning models by minimizing training loss.  This can work very well, but there are many instances where making the training loss as small as possible can actually cause the model's performance on validation data (i.e., the validation loss) *to get worse* (see the figure below)!   This problem is referred to as overtraining or overfitting.  Overfitting occurs when the model starts to fit to characteristics of the training set that don't generalize to new data.


![Two loss function](../nb-images/Regularization.svg)
<div style="text-align: right"> (Image from Google machine learning crash course) </div>

For instance, consider this simple dataset that contains information about insect, fish, and bird species and whether or not they can fly:

|Name|Class|Can fly|
|:--:|:---:|:-----:|
|Pileated woodpecker|Birds|Yes|
|Emu|Birds|No|
|Northern cardinal|Birds|Yes|
|Blacktip shark|Cartilaginous fishes|No|
|Bluntnose stingray|Cartilaginous fishes|No|
|Black drum|Bony fishes|No|
|Florida carpenter ant|Insects|No|
|Periodical cicada|Insects|Yes|
|Luna moth|Insects|Yes|

Your task is to develop a model to classify whether or not an animal can fly, based on information available in the dataset.  Here is a relatively simple model based on these data:
  * If the animal is a bird or an insect, predict that it can fly.
  * Otherwise, predict that it cannot fly.

This model is imperfect.  Indeed, it misclassifies 2 of the examples in the training data.  We can try developing a more complex model to reduce our training loss:
  * If the species is a bird and has a one-word name, predict that it cannot fly.
  * If it is a bird with a two-word name, predict that it can fly.
  * If it is an insect with a three-word name, predict that it cannot fly.
  * If it is an insect with a two-word name, predict that it can fly.
  * Otherwise, predict that it cannot fly.

Aha!  That model classifies each training example perfectly!

When presented with new examples, however (e.g., "albatross" or "zebra swallowtail butterfly"), the more complex model will often fail spectacularly while the simpler model, although imperfect, will perform relatively well.  Clearly, our more complex model is disastrously overfitted to the training data.

The bottom line is that we want our models to be general enough to work well on new examples.  Methods to help prevent overfitting are collectively referred to as *regularization* techniques.  Another way to think about regularization is that you should not trust your training examples too much.

There are a variety of ways to implement regularization to prevent overfitting.  One option is to stop training early before overfitting happens. In the figure above, you would try to stop at the point on the red curve where validation loss begins to increase.  Another approach, which is often easier to implement in practice, is to explicitly penalize model complexity in the model-fitting procedure.  Instead of just minimizing loss, we minimize `loss + complexity`.

## 2. L<sub>1</sub> and L<sub>2</sub> regularization

For this lesson, we will focus on two widely used regularization methods: L<sub>1</sub> and L<sub>2</sub> regularization.  Both of these methods represent model complexity as a function of the model's feature weights.

Suppose we are fitting a general linear regression model, which looks like this:

$$ y = w_0 + w_1 x_1 + w_2 x_2 + \ldots + w_k x_k $$

L<sub>1</sub> regularization defines model complexity as the sum of the absolute value of the feature weights (multiplied by a constant, $\lambda$, which we will discuss later).

$$L_1\text{ }regularization\text{ }penalty = \lambda\sum_{i=1}^k |w_i|$$

In this formula, weights near zero have small effects on the model complexity, where larger weights have a larger effect. 

For example, if your model has the weights `[-0.5, -0.2, 0.5, 0.7, 1.0, 2.5]`, the L<sub>1</sub> regularization term is just the sum of absolute value of each of the weights:

In [ ]:
import numpy as np
weights = [-0.5, -0.2, 0.5, 0.7, 1.0, 2.5]
print(sum(np.absolute(weights)))

L<sub>2</sub> regularization represents model complexity as the sum of the squares of the feature weights (again multiplied by a constant, $\lambda$).

$$L_2\text{ }regularization\text{ }penalty = \lambda\sum_{i=1}^k w_i^2$$

Using the same weights from above, the L<sub>2</sub> regularization term can be computed with the following code.

In [ ]:
print(sum(np.square(weights))) 

### 2.a. Adding regularization to a loss/cost function

To use either of these regularization terms when fitting a model, the usual procedure is to add the regularization term to whatever loss function you want to use.  E.g., *total loss* = *loss* + *regularization penalty*.  Let's look at an example using linear regression.  Recall that the usual loss function for linear regression is the *mean square error*:

$$ MSE = \frac{1}{n} \sum_{i=1}^n (y_i - (w_0 + w_1 x_{i,1} + w_2 x_{i,2} + \ldots + w_k x_{i,k}))^2 $$

Then, we add the regularization penalty to the loss function.  In the case of L<sub>1</sub> regularization, for instance, we would have:

$$ MSE + \lambda\sum_{i=1}^k |w_i|$$



### 2.b. Lambda

The regularization parameter in the above formulas, $\lambda$, allows you to adjust the balance between minimizing the loss function and penalizing overly complex models.  Increasing $\lambda$ strengthens the regularization effect and will cause more of the model weights to be near zero, while decreasing $\lambda$ places more emphasis on reducing the loss function.  Thus, the value of $\lambda$ is very important during model fitting.  Too large a value for $\lambda$ and your model might be overly simple and prone to underfitting.  Too small of a value and your model will be more complex, but you run the risk of overfitting. The optimal value of lambda is data-dependent and will usually need to be estimated in some way.


### 2.c. Practical differences between L<sub>1</sub> and L<sub>2</sub> regularization

L<sub>1</sub> and L<sub>2</sub> both can help prevent overfitting.  From a practical standpoint, perhaps the most important difference between the two is that L<sub>1</sub> regularization can help with *feature selection*.  As a consequence of the mathematical properties of L<sub>1</sub> regularization, L<sub>1</sub> regularization can result in models where some of the feature weights are 0, effectively removing those features from the model.  In contrast, L<sub>2</sub> regularization can decrease model weights but not drive them to 0.  L<sub>1</sub> regularization can also be more robust and resistant to larger outliers in the data.  On the other hand, L<sub>2</sub> regularization results in a minimization problem with a unique solution, which is not always the case for L<sub>1</sub> regularization.  Which regularization method is best depends on the specifics of the data, the modeling problem, and the goals of the analysis.
 

## 3. Practice example / demonstration

Now, let's use scikit-learn to take a look at these ideas in actual practice.  We'll start by working with a dataset called `regularization.csv` that you can find in the `nb-datasets` folder.

### Step 1: Import the required libraries and load the data

In [62]:
import pandas as pd
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse

rdata = pd.read_csv('../nb-datasets/regularization.csv')
rdata.head()

,response,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30
0,72.456984,32.321012,65.689328,52.122727,63.399651,12.258814,17.043635,51.730222,62.860680,2.872635,...,76.858001,75.087967,38.094935,17.919735,37.380254,66.538720,23.598782,35.905618,43.137890,47.112709
1,28.327361,30.668730,54.556984,58.913683,24.094272,69.364714,17.979578,56.464967,84.401995,51.842014,...,33.629986,75.302974,36.735844,56.038455,26.910583,94.038582,63.119873,48.105001,47.805156,73.564217
2,57.966809,26.057286,26.943960,28.413263,47.659902,58.102919,60.523707,33.314868,33.338900,37.278545,...,45.392581,45.232257,93.975501,27.220155,39.191009,17.016580,47.965641,33.711634,69.566848,83.822185
3,45.432275,33.974760,49.752550,37.456331,38.234778,41.094095,15.415567,61.188945,70.176046,26.425137,...,29.434284,47.596757,93.733349,33.764953,46.434659,51.489927,60.676096,48.719651,77.244083,46.888295
4,41.635332,12.159012,60.621783,58.770553,35.291748,37.930786,35.609174,24.067184,29.361941,66.153336,...,69.537131,48.294237,85.607670,25.917772,35.023185,86.978918,24.345884,67.140613,21.886626,21.017448


### Step 2: Prepare the training and testing datasets

In [81]:
# Separate the x and y values.
x = rdata.drop(columns='response')
y = rdata['response']

# Split the train and test sets.  Use 25% of the data for testing.
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.5, random_state=7
)
len(rdata)

64

### Step 3: Fit a standard linear regression model

We'll give regular old non-regularized linear regression a try first.  As a reminder, we'll fit the model using the training data and then check how it performs on the testing data.

In [82]:
model = LinearRegression()
model.fit(x_train, y_train)

# Take a look at the model's R^2 score and training loss (mean squared error).
print('Train R2:', model.score(x_train, y_train))
train_loss = mse(y_train, model.predict(x_train))
print('Train loss:', train_loss)

# Let's also have a look at the model's coefficients.
print('\nCoefficients:', model.coef_)

Train R2: 0.9999392834150598
Train loss: 0.029419148916598636

Coefficients: [-4.12797020e-02 -1.79616277e-01 -4.99773043e-02  1.17300976e+00
  2.82296227e-02  2.27359809e-01 -7.21474264e-03  5.07660342e-02
 -1.35590182e-01  1.09717013e-01 -1.72193496e-01 -2.14314632e-01
  1.11973445e-01 -4.24759522e-02 -1.12168799e-01 -8.45964728e-02
  5.65948007e-02 -1.88272052e-01  3.31274473e-01 -2.79209727e-02
  1.16344162e-02  4.69337409e-02  2.38795254e-02 -1.90923687e-02
  6.35118218e-04 -6.35080596e-02  3.56423726e-02  5.60962269e-02
  4.40148056e-02  3.75539252e-02]


### Step 4: Check the model's performance on the testing data

Our model predicts the training responses extremely well (almost perfectly!).  Let's see how it does on the testing data.

In [83]:
print('Test R2:', model.score(x_test, y_test))

test_loss = mse(y_test, model.predict(x_test))
print('Test loss:', test_loss)

Test R2: 0.49778230757655045
Test loss: 199.7198626958782


Uh oh.  Predictions on "new" data are terrible!  This result indicates that our model is probably badly overfit to the training data.


### Step 5: Try using L<sub>1</sub> regularization

Let's see if we can use L<sub>1</sub> regularization to fix this.  In scikit-learn, the `Lasso` object provides linear regression with L<sub>1</sub> regularization.  We use it in much the same way as `LinearRegression`, but we need to provide a value for the parameter `alpha`, which corresponds with the parameter $\lambda$ discussed in the regularization formulas above.

In [86]:
model = Lasso(alpha=22.0)
model.fit(x_train, y_train)

print('Train R2:', model.score(x_train, y_train))
train_loss = mse(y_train, model.predict(x_train))
print('Train loss:', train_loss)

print('\nTest R2:', model.score(x_test, y_test))
test_loss = mse(y_test, model.predict(x_test))
print('Test loss:', test_loss)

print('\nCoefficients:', model.coef_)

Train R2: 0.9897749936281043
Train loss: 4.954346253562866

Test R2: 0.9865604019011951
Test loss: 5.344603998375144

Coefficients: [ 0.         -0.          0.          1.11487814  0.          0.
 -0.         -0.         -0.         -0.          0.         -0.
  0.         -0.          0.         -0.          0.          0.
  0.          0.         -0.          0.         -0.         -0.
 -0.         -0.         -0.         -0.         -0.          0.        ]


Great success!  As is typical with regularization, the model's performance on the training data has decreased a little bit, but its performance on the testing data has massively improved.  The model's performance on "new" data is now competitive with its performance on the training data, which is about the best we can hope for.

Note also that with a regularization parameter (`alpha`/$\lambda$ ) of 22.0, we've applied fairly aggressive regularization to our model fit, which has resulted in the L<sub>1</sub> regularization driving *nearly all* of the model's coefficients/feature weights to 0!  Only the coefficient for `feature_4` remains.  In other words, the L<sub>1</sub> regularization has drastically simplified our model by eliminating all features except for `feature_4`.

Does that result make sense?  As it turns out, it does.  The dataset we've been using is an artificial dataset that I created for this exercise, so we know the true relationship of the features to the response.  As suggested by the regularized coefficient estimates, `feature_4` is the only feature that has any relationship with the response variable.  In reality, `response` is equal to `feature_4` * 1.2 plus a small amount of random noise.  *All* of the other features are entirely random and have no relationship to `response` at all.

Although this is a contrived dataset, it is not as unrealistic as it might at first seem.  With complex real-world data, we have often have a large number of candidate features/predictor variables, and we don't know which, if any, of those features have any real relationship with the response variable.  In those situations, overfitting is a real concern, especially if the number of observations is relatively small, and regularization can help prevent us from drawing the wrong conclusions from our data.

What about the value of the regularization parameter, `alpha`/$\lambda$?  Here, I deliberately chose a value (20.0) that I knew would work well given the truth behind the dataset.  With real data, we never have that luxury, and more sophisticated methods (that we don't have time to present in this short course) are required to select an appropriate value of $\lambda$.


### Exercise

Try experimenting with the value of the regularization parameter in the code above.  How does changing the value of alpha affect the results?  When do you get results that are misleading or just plain wrong?


### Step 6: Try using L<sub>2</sub> regularization

We can also analyze these data using L<sub>2</sub> regularization.  To do that, we'll use a scikit-learn object called `Ridge` that implements linear regression with L<sub>2</sub> regularization.  The interface of `Ridge` is exactly the same as the interface of `Lasso`.

In [ ]:
model = Ridge(alpha=1.0)
model.fit(x_train, y_train)

print('Train R2:', model.score(x_train, y_train))
train_loss = mse(y_train, model.predict(x_train))
print('Train loss:', train_loss)

print('\nTest R2:', model.score(x_test, y_test))
test_loss = mse(y_test, model.predict(x_test))
print('Test loss:', test_loss)

print('\nCoefficients:', model.coef_)

This illustrates a key difference between L<sub>1</sub> regularization and L<sub>2</sub> regularization: L<sub>1</sub> regularization is able to drive feature weights/model coefficients to 0 and effectively remove features from a model, but L<sub>2</sub> regularization cannot, even when doing so would be appropriate.

Consequently, for these data, L<sub>2</sub> regularization helps a bit with overfitting, but it clearly does not perform nearly as well as L<sub>1</sub> regularization.  That won't always be the case, though.  This dataset is an intentionally extreme examle that suits L<sub>1</sub> regularization especially well.  For datasets where all or most of the features really do have some connection to the response or where collinearity is a concern, L<sub>2</sub> regularization might be a better choice.


## 4.  Practice example using real data

As a final example, let's try using regularization on a real dataset.  We'll again use the iris dataset that you've already seen in previous lessons.  We might not have time for this example during the workshop, and if not, I encourage you to explore it on your own.

### Steps 1 and 2: Load the data and split out training and testing sets.

In [ ]:
idata = pd.read_csv('../nb-datasets/iris_dataset.csv')
idata['species'] = idata['species'].astype('category')

# Convert the categorical variable "species" to 1-hot encoding (AKA "dummy variables"),
# but eliminate the first dummy variable because it is collinear with the other two
# and does not provide any additional information.
idata_enc = pd.get_dummies(idata, drop_first=True)

# Separate the x and y values.
x = idata_enc.drop(columns='petal_length')
y = idata_enc['petal_length']

# Split the train and test sets.
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

# See what we have.
idata_enc.head()

### Steps 3 and 4: Give standard linear regression a try

We'll fit a standard linear regression model and check its performance on the testing data.  This will give us a baseline to compare to the regularization methods.

In [ ]:
model = LinearRegression()
model.fit(x_train, y_train)

print('Train R2:', model.score(x_train, y_train))
train_loss = mse(y_train, model.predict(x_train))
print('Train loss:', train_loss)

print('\nTest R2:', model.score(x_test, y_test))
test_loss = mse(y_test, model.predict(x_test))
print('Test loss:', test_loss)

print('Coefficients:', model.coef_)

### Step 5: L<sub>1</sub> regularization

The results above *strongly* suggest that overfitting really isn't much of a problem here.  We have a relatively small number of features (i.e., parameters we must estimate) compared to the number of observations, so that result makes sense.  Nevertheless, let's try fitting the model using L<sub>1</sub> regularization and see what happens.


In [ ]:
model = Lasso(alpha=0.01)
model.fit(x_train, y_train)

print('Train R2:', model.score(x_train, y_train))
train_loss = mse(y_train, model.predict(x_train))
print('Train loss:', train_loss)

print('\nTest R2:', model.score(x_test, y_test))
test_loss = mse(y_test, model.predict(x_test))
print('Test loss:', test_loss)

print('\nCoefficients:', model.coef_)

### Step 6: L<sub>2</sub> regularization

In [ ]:
model = Ridge(alpha=2.0)
model.fit(x_train, y_train)

print('Train R2:', model.score(x_train, y_train))
train_loss = mse(y_train, model.predict(x_train))
print('Train loss:', train_loss)

print('\nTest R2:', model.score(x_test, y_test))
test_loss = mse(y_test, model.predict(x_test))
print('Test loss:', test_loss)

print('\nCoefficients:', model.coef_)

### Exercises

Try experimenting with the value of `alpha`/$\lambda$ in the code above for both L<sub>1</sub> regularization and L<sub>2</sub> regularization.  As you do so, consider these questions:

1. How does changing the value of the regularization parameter affect the coefficient weights and training/test performance?
2. What values of the regularization parameter give you the best test accuracy?
3. For these data, does L<sub>1</sub> or L<sub>2</sub> regularization perform better?